In [15]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import os

In [16]:
os.getcwd()

'/Users/haydenprescott/projects/red-flag-text-detection/data/sources/set2'

In [25]:
df = pd.read_csv("./DepressionDetectionNLP/final_training_data.txt", delimiter="\t")
# Rename column 'combined' to 'text' for better clarity
df.rename(columns={'combined':'text'}, inplace=True)

# Add columns for positive/negative sentiment and the label
df["label"] = ""
df["pos"] = ""
df["neg"] = ""

df.head()

,text,subreddit,label,pos,neg
0,Need to Change Depression meds....but SCARED.....,sw,,,
1,Fear of surviving stopping me from kms If ther...,sw,,,
2,"Going through major transformations I’m 25, fr...",cc,,,
3,I’ve started having suicidal thoughts again I’...,sw,,,
4,Who would you choose?! So imagine one day you ...,cc,,,


In [27]:
#Perform Sentiment Analysis on Training Data and Populate Pandas DataFrame
analyzer = SentimentIntensityAnalyzer()

# Thresholds to discard outliers (positive posts in suicide watch subreddit & negative posts in casual conversation subreddit)
NEG_DISCARD_THRESHOLD = 0.1
POS_DISCARD_THRESHOLD = 0.1

for index, row in df.iterrows():
    subreddit = row.subreddit
    
    # Run sentiment anlysis for the post
    scores = analyzer.polarity_scores(row.text)
    
    if scores['neg'] > NEG_DISCARD_THRESHOLD and subreddit == "cc":
        df.drop(index, inplace=True)
    elif scores['pos'] > POS_DISCARD_THRESHOLD and subreddit == "sw":
        df.drop(index, inplace=True)
    else:
        if subreddit == "cc":
            row.label = 1
        elif subreddit == "sw":
            row.label = 0
        else:
            row.label = np.nan
        row.pos = scores['pos']
        row.neg = scores['neg']
df.head()

,text,subreddit,label,pos,neg
0,Need to Change Depression meds....but SCARED.....,sw,0,0.02,0.249
6,Where should I read my college required readin...,cc,1,0.0,0.0
7,"Your and You’re An easy way to remember it, fo...",cc,1,0.078,0.066
8,"did i miss a day? oh poop. well, halfway to 5 ...",cc,1,0.353,0.028
9,My flight just got delayed for super long.. wh...,cc,1,0.065,0.032


In [28]:
df.to_csv("./set2_processed.csv")